## Final project

In [1]:
import facebook_scraper as fs
import pandas as pd
import datetime
import numpy as np
import os
import time
from facebook_scraper import _scraper

/home/vnn04/miniconda3/envs/ml-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
FANPAGE_LINK = "sieuanhhungMarvel"
current_dir = os.getcwd()
FOLDER_PATH = current_dir + "/Data/"
print(FOLDER_PATH)

/home/vnn04/Documents/facebook_scraper_page/Data/


#### Nhập tài khoản và mật khẩu facebook

In [3]:
user_email = "0563656200"
user_password = "34md506647"

#### Set cookies

In [4]:
_scraper.login(user_email, user_password)
cookies = _scraper.session.cookies
# Store & retrieve cookies
fs.set_cookies(cookies)
# Check it worked
print(_scraper.is_logged_in())

/home/vnn04/miniconda3/envs/ml-env/lib/python3.12/site-packages/facebook_scraper/facebook_scraper.py:855: UserWarning: Facebook language detected as vi_VN - for best results, set to en_US
  warnings.warn(


True


In [5]:
day_start = datetime.datetime(2023, 11, 19, 0, 0, 0)
day_end = datetime.datetime(2023, 9, 19, 0, 0, 0)

#### Lấy thông tin của các bài viết trong 2 tháng vừa qua (từ 19/9/2023 - 19/11/2023)

In [14]:
post_list = []

for post in fs.get_posts(FANPAGE_LINK,
                    options={"comments": False, "reactions": False, "allow_extra_requests": False},
                    pages=1000):
    if post['time'] > day_start:
        continue
    else:
        if post['time'] <= day_end:
            break
        print(post)
        post_list.append({'post_id':post['post_id'], 'text':post['text'], 'time':post['time'], 'reactions':post['reactions'], 'reaction_count':post['reaction_count']})


{'post_id': '734518898719448', 'text': 'Các bạn nam chắc ai cũng muốn học chiêu pause time này để dành khi cãi nhau với người yêu :))', 'post_text': 'Các bạn nam chắc ai cũng muốn học chiêu pause time này để dành khi cãi nhau với người yêu :))', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2023, 11, 18, 22, 40, 26), 'timestamp': 1700322026, 'image': None, 'image_lowquality': 'https://scontent.fhan14-4.fna.fbcdn.net/v/t39.30808-6/402264624_734518948719443_5938829508463439003_n.jpg?stp=cp0_dst-jpg_e15_p160x160_q65&_nc_cat=103&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=NRb9YeaNfLQAX87bQAf&_nc_ht=scontent.fhan14-4.fna&oh=00_AfDAQ3TeMsdn-2JYe1MPu7zBzxEweBWhv06gBlsC86b5aA&oe=6560B153', 'images': None, 'images_description': None, 'images_lowquality': ['https://scontent.fhan14-4.fna.fbcdn.net/v/t39.30808-6/402264624_734518948719443_5938829508463439003_n.jpg?stp=cp0_dst-jpg_e15_p160x160_q65&_nc_cat=103&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=NRb9YeaNfLQAX87bQ

#### Số lượng post lấy được từ 19/9/2023 đến 19/11/2023

In [17]:
print(len(post_list))

216


#### Lưu danh sách thông tin cơ bản của các post vào file sieuanhhungMarvel.csv

In [18]:
import csv

path = FOLDER_PATH + FANPAGE_LINK + ".csv"

with open(path, 'w', newline='', encoding='utf-8') as csv_file:
    # Tạo đối tượng csv.writer
    csv_writer = csv.writer(csv_file)

    # Ghi header
    header = post_list[0].keys()
    csv_writer.writerow(header)

    # Ghi dữ liệu
    for row in post_list:
        csv_writer.writerow(row.values())


#### Đọc thông tin cơ bản của 5 post đầu (tính từ ngày 19/11/2023)

In [20]:
df = pd.read_csv('Data/sieuanhhungMarvel.csv')
df.head()

,post_id,text,time
0,734518898719448,Các bạn nam chắc ai cũng muốn học chiêu pause ...,2023-11-18 22:40:26
1,734285772076094,Nếu như Pedro một mình cân tất cả các vai ở Fa...,2023-11-18 12:51:48
2,734234215414583,Sau bom tấn Spider-Man 2 thì dự án tiếp theo c...,2023-11-18 10:21:01
3,734194055418599,Infinity Magic 🔥,2023-11-18 08:30:34
4,733990162105655,Nam diễn viên Steven Yeun được xác nhận từ nhi...,2023-11-17 23:17:50


#### Đọc post_id từ file sieuanhhungMarvel.csv

In [41]:
path = FOLDER_PATH + FANPAGE_LINK + ".csv"

In [42]:
df_post = pd.read_csv(path)

df_post_id = df_post['post_id']
df_post_id.head(10)

0    734518898719448
1    734285772076094
2    734234215414583
3    734194055418599
4    733990162105655
5    733860328785305
6    733772145460790
7    733681738803164
8    733428062161865
9    733305832174088
Name: post_id, dtype: int64

#### Lấy tất cả id của các post cần crawl dữ liệu

In [22]:
post_id = df_post_id.to_numpy()

#### Lấy reactions, comments, shares, comments_full

In [46]:
def get_reactions_comments_full_shares(url):
    info = []
    for data in fs.get_posts(post_urls=[url], options={'reactions':True, 'comments':True}):
        info.append({'reactions':data['reactions'], 'reaction_count':data['reaction_count'], 'shares':data['shares'], 'comments':data['comments'], 'comments_full':data['comments_full']})
    return info

def get_more_info(url):
    data = get_reactions_comments_full_shares(url)

    if data is None or not data:
        return 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

    if 'reactions' == None:
        like, love, haha, sad, wow, care, angry = 0, 0, 0, 0, 0, 0, 0
    else:
        reactions = data[0]['reactions']
        if 'like' not in reactions:
            like = 0
        else:
            like = reactions['like']
        if 'love' not in reactions:
            love = 0
        else:
            love = reactions['love']
        if 'haha' not in reactions:
            haha = 0
        else:
            haha = reactions['haha']
        if 'wow' not in reactions:
            wow = 0
        else:
            wow = reactions['wow']
        if 'sad' not in reactions:
            sad = 0
        else:
            sad = reactions['sad']
        if 'care' not in reactions:
            care = 0
        else:
            care = reactions['care']
        if 'angry' not in reactions:
            angry = 0
        else: 
            angry = reactions['angry']

    reaction_count = data[0].get('reaction_count', 0)
    shares = data[0].get('shares', 0)
    comment = data[0].get('comments', 0)
    comments_full = data[0].get('comments_full', [])

    return like, love, haha, sad, wow, care, angry, reaction_count, shares, comment, comments_full


#### Thêm cột like, love, haha, care, sad, angry, reaction_count, comments, comments_full, shares

In [ ]:
like_list = []
love_list = []
haha_list = []
sad_list = []
wow_list = []
care_list = []
angry_list = []
reaction_count_list = []
shares_list = []
comment_list = []
comments_full_list = []

for id in post_id:
    like, love, haha, sad, wow, care, angry, reaction_count, shares, comment, comments_full = get_more_info(id)
    like_list.append(like)
    love_list.append(love)
    haha_list.append(haha)
    sad_list.append(sad)
    wow_list.append(wow)
    care_list.append(care)
    angry_list.append(angry)
    reaction_count_list.append(reaction_count)
    shares_list.append(shares)
    comment_list.append(comment)
    comments_full_list.append(comments_full)
    time.sleep(2)

In [ ]:
df_post['like'] = like_list
df_post['love'] = love_list
df_post['haha'] = haha_list
df_post['sad'] = sad_list
df_post['wow'] = wow_list
df_post['care'] = care_list
df_post['angry'] = angry_list
df_post['reaction_count'] = reaction_count_list
df_post['shares'] = shares_list
df_post['comment'] = comment_list
df_post['comments_full'] = comments_full_list

#### Lấy thông tin cơ bản của reactors

In [ ]:
def get_all_reactors(url):
    reactors = []
    for reactor in fs.get_posts(post_urls=url, options={'reactors':True}):
        print(reactor['reactors'])
        reactors.append(reactor)
        time.sleep(0.1)
    return reactors

####

In [34]:
a = reactor_id[1]
print(a)

IndexError: list index out of range

In [31]:
print(a['link'])

https://facebook.com/profile.php?id=100095122470934&eav=AfaJYf7oYWzs_aN5rutrgqOoTcamiLxFqL45iQEFDF7_-bWnrpKMbOy0OWrvVxYupxk&fref=pb&paipv=0


In [45]:
text = fs.get_profile('troi2004day')

/home/vnn04/miniconda3/envs/ml-env/lib/python3.12/site-packages/facebook_scraper/facebook_scraper.py:908: UserWarning: Facebook served mbasic/noscript content unexpectedly on https://m.facebook.com/timeline/app_collection/?collection_token=100013048213229%3A103382489711013%3A33&paipv=0&eav=AfZUNq8OlQ8UPqfk-gAfPO4yB9FGAhgKsVkQ3VZ7BYR-olY5hsbljIjLYNyFr46qzVU&_rdr
  warnings.warn(


{'top_post': {'post_id': '1455738051537788', 'text': 'Hãy nhớ rằng: "Nếu thực sự muốn thì người ta sẽ tìm cách, còn nếu không muốn thì người ta sẽ tìm lý do."', 'post_text': 'Hãy nhớ rằng: "Nếu thực sự muốn thì người ta sẽ tìm cách, còn nếu không muốn thì người ta sẽ tìm lý do."', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2022, 7, 14, 6, 33), 'timestamp': None, 'image': None, 'image_lowquality': 'https://scontent.fhan4-3.fna.fbcdn.net/v/t39.30808-6/292428127_5485895144829908_7923343886320211679_n.png?stp=dst-png_s320x320&_nc_cat=103&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=UYEqCdrxttUAX9khrtM&_nc_ht=scontent.fhan4-3.fna&oh=00_AfDnyzA7ZFjc0H4l3fHFlXhWiTclnmQYkb-VY5YwJ4lu2A&oe=655653A8', 'images': [], 'images_description': [], 'images_lowquality': ['https://scontent.fhan4-3.fna.fbcdn.net/v/t39.30808-6/292428127_5485895144829908_7923343886320211679_n.png?stp=dst-png_s320x320&_nc_cat=103&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=UYEqCdrxttUAX9khrtM&_n

In [46]:
text['Thông tin cơ bản']

'17 tháng 10, 2003\nNgày sinh\nNam\nGiới tính'

In [47]:
split_text = text['Thông tin cơ bản'].split('\n')
ngaysinh = split_text[0]
gioitinh = split_text[2]

print(ngaysinh, '\n', gioitinh)

17 tháng 10, 2003 
 Nam
